<a href="https://colab.research.google.com/github/ManoharBandam/PSU/blob/master/tapas_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/bhattbhavesh91/table-question-answering-demo/blob/main/table-qa-pipeline.ipynb
# module 1 tapas

In [2]:
!pip install -q transformers==4.4.2


In [3]:
!pip install sentencepiece

In [4]:
import torch
torch.__version__

'1.11.0+cu113'

In [5]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html


Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html


In [6]:
from transformers import pipeline
import pandas as pd


In [7]:
tqa = pipeline(task="table-question-answering", 
               model="google/tapas-base-finetuned-wtq")

In [8]:
df1 = pd.read_json('sample_data/fetaQA-v1_train.json')
df2 = pd.json_normalize(df1['data'])
# df2.columns = df2.columns.map(lambda x: x.split(".")[-1])
# df3 = df2['data'].to_csv()
# df3 = df2.astype(str)

In [9]:
df2.head()

,feta_id,table_source_json,page_wikipedia_url,table_page_title,table_section_title,table_array,highlighted_cell_ids,question,answer,source
0,12019,totto_source/train_json/example-4318.json,http://en.wikipedia.org/wiki/Core_fonts_for_th...,Core fonts for the Web,List of fonts and files,"[[File name, Font name, Variants, Last version...","[[1, 1], [1, 3], [9, 1], [9, 3], [20, 1], [20,...",What were the font-versions that were availabl...,The font-versions that were available from Cor...,mturk-approved
1,10691,totto_source/train_json/example-2990.json,http://en.wikipedia.org/wiki/Shruti_Haasan_fil...,Shruti Haasan filmography,Filmography,"[[Title, Year, Role(s), Director, Language(s),...","[[1, 0], [1, 1], [1, 4], [1, 5], [2, 0], [2, 1...",In what films has Shruti Haasan made a cameo a...,Shruti Haasan made a cameo appearance in Tamil...,internal
2,8300,totto_source/train_json/example-599.json,http://en.wikipedia.org/wiki/List_of_Assyrian_...,List of Assyrian kings,Dynasty of Puzur-Ashur (2025–1749 BC),"[[#, Image, King, Reign, Succession, Notes, Re...","[[5, 2], [6, 2], [6, 4], [7, 2], [7, 4]]","On the Assyrian King List, for whom is Sargon ...","On the Assyrian King List, Sargon appears as t...",internal
3,8107,totto_source/train_json/example-406.json,http://en.wikipedia.org/wiki/List_of_Stanley_C...,List of Stanley Cup champions,Active teams,"[[Apps, Team, Wins, Losses, Win %, Years of ap...","[[1, 1], [1, 2], [3, 1], [3, 2]]",How many Stanley Cup championships did Toronto...,The Toronto Maple Leafs has won thirteen Stanl...,internal
4,10564,totto_source/train_json/example-2863.json,http://en.wikipedia.org/wiki/Estadio_El%C3%ADa...,Estadio Elías Aguirre,International matches hosted,"[[Date, Team #1, Res., Team #2, Round, Attenda...","[[1, 1], [1, 2], [1, 3], [1, 4], [5, 1], [5, 2...",What were the first and final Copa América mat...,The game that opened the Elías Aguirre's parti...,internal


In [10]:
df2.count()

feta_id                 7325
table_source_json       7325
page_wikipedia_url      7325
table_page_title        7325
table_section_title     7325
table_array             7325
highlighted_cell_ids    7325
question                7325
answer                  7325
source                  7325
dtype: int64

In [11]:
table_df = pd.DataFrame(df2["table_array"][0])
# ,3print(table_df)
table_df.iloc[0,3]
# table_df.columns[3]
# for col in table_df.columns:
#     print(col)

'Last version'

In [39]:
dart_df = pd.DataFrame()
for i in range(1000):
  if i%50 == 0:
    print("running epoch:",i)
  table_df = pd.DataFrame(df2["table_array"][i])
  label = df2["answer"][i]
  table_df.columns = table_df.iloc[0] 
  table_df1 = table_df[1:]
  query1 = str(df2["question"][i])
  # print(query1)
  tqa_result = tqa(table=table_df, query=query1)
  # print(tqa_result)
  # print(tqa_result.get("coordinates"))
  # print(len(tqa_result.get("coordinates")))
  tqa_result["table_name"] = df2['table_page_title'][i]
  tqa_result["table_section_title"] = df2['table_section_title'][i]
  tqa_result["target"] = label
  # print(table_df.iloc[0,3])
  tqa_result["column_header"] = table_df.iloc[0,int(tqa_result.get("coordinates")[0][1])] if len(tqa_result.get("coordinates"))>0 else " "
  dart_df = dart_df.append(tqa_result, ignore_index=True)


In [41]:
dart_df.count()

answer                 1000
coordinates            1000
cells                  1000
aggregator             1000
table_name             1000
table_section_title    1000
target                 1000
column_header          1000
dtype: int64

In [40]:
dart_df.head()

,answer,coordinates,cells,aggregator,table_name,table_section_title,target,column_header
0,"version 1.03, version 1.01","[(28, 3), (29, 3)]","[version 1.03, version 1.01]",NONE,Core fonts for the Web,List of fonts and files,The font-versions that were available from Cor...,Last version
1,"Hey Ram, Hey Ram","[(1, 0), (2, 0)]","[Hey Ram, Hey Ram]",NONE,Shruti Haasan filmography,Filmography,Shruti Haasan made a cameo appearance in Tamil...,Title
2,"Sargon I Šarru-ukīn ""the king is legitimate""","[(6, 2)]","[Sargon I Šarru-ukīn ""the king is legitimate""]",NONE,List of Assyrian kings,Dynasty of Puzur-Ashur (2025–1749 BC),"On the Assyrian King List, Sargon appears as t...",King
3,"COUNT > 24, 13","[(1, 2), (3, 2)]","[24, 13]",COUNT,List of Stanley Cup champions,Active teams,The Toronto Maple Leafs has won thirteen Stanl...,Wins
4,"7 July 2004, 7 July 2004, 10 July 2004, 10 Jul...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0)]","[7 July 2004, 7 July 2004, 10 July 2004, 10 Ju...",NONE,Estadio Elías Aguirre,International matches hosted,The game that opened the Elías Aguirre's parti...,Date


In [42]:
dart_df_filtered = dart_df.copy()
dart_df_filtered["table_context"] = dart_df_filtered["table_section_title"]+" in "+dart_df_filtered["table_name"]
dart_df_filtered = dart_df_filtered.drop(['coordinates', 'aggregator','table_name','table_section_title','cells'], axis = 1)

In [43]:
dart_df.head()

,answer,coordinates,cells,aggregator,table_name,table_section_title,target,column_header
0,"version 1.03, version 1.01","[(28, 3), (29, 3)]","[version 1.03, version 1.01]",NONE,Core fonts for the Web,List of fonts and files,The font-versions that were available from Cor...,Last version
1,"Hey Ram, Hey Ram","[(1, 0), (2, 0)]","[Hey Ram, Hey Ram]",NONE,Shruti Haasan filmography,Filmography,Shruti Haasan made a cameo appearance in Tamil...,Title
2,"Sargon I Šarru-ukīn ""the king is legitimate""","[(6, 2)]","[Sargon I Šarru-ukīn ""the king is legitimate""]",NONE,List of Assyrian kings,Dynasty of Puzur-Ashur (2025–1749 BC),"On the Assyrian King List, Sargon appears as t...",King
3,"COUNT > 24, 13","[(1, 2), (3, 2)]","[24, 13]",COUNT,List of Stanley Cup champions,Active teams,The Toronto Maple Leafs has won thirteen Stanl...,Wins
4,"7 July 2004, 7 July 2004, 10 July 2004, 10 Jul...","[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0)]","[7 July 2004, 7 July 2004, 10 July 2004, 10 Ju...",NONE,Estadio Elías Aguirre,International matches hosted,The game that opened the Elías Aguirre's parti...,Date


In [44]:
dart_df_filtered.head()

,answer,target,column_header,table_context
0,"version 1.03, version 1.01",The font-versions that were available from Cor...,Last version,List of fonts and files in Core fonts for the Web
1,"Hey Ram, Hey Ram",Shruti Haasan made a cameo appearance in Tamil...,Title,Filmography in Shruti Haasan filmography
2,"Sargon I Šarru-ukīn ""the king is legitimate""","On the Assyrian King List, Sargon appears as t...",King,Dynasty of Puzur-Ashur (2025–1749 BC) in List ...
3,"COUNT > 24, 13",The Toronto Maple Leafs has won thirteen Stanl...,Wins,Active teams in List of Stanley Cup champions
4,"7 July 2004, 7 July 2004, 10 July 2004, 10 Jul...",The game that opened the Elías Aguirre's parti...,Date,International matches hosted in Estadio Elías ...


In [66]:
dart_df_filtered["target"][1]

'Shruti Haasan made a cameo appearance in Tamil-Hindi bilingual directorial Hey Ram (2000).'

In [55]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [56]:
batch_size=8
num_of_batches=len(dart_df_filtered)/batch_size

In [57]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

In [59]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)


In [60]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
input = dart_df_filtered["answer"][0]
label = dart_df_filtered["target"][0]
# print(tokenizer)
tokenizer.batch_encode_plus(input,padding=True,max_length=400,return_tensors='pt')["input_ids"]

In [62]:
#Sets the module in training mode
model.train()
num_of_epochs = 4
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in range(int(num_of_batches)):
    inputbatch=[]
    labelbatch=[]
    new_df=dart_df_filtered[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      # input = 'WebNLG: '+row['input_text']+'</s>'
      input = 'WebNLG: '+row['answer']+'</s>'+row['column_header']+'</s>'+row['table_context']+'</s>' 
      labels = row['target']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
    inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    if i%10 ==0:      
      loss_per_10_steps.append(loss_num)
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Epoch: 1 , Running loss: 2.0413947010040285
Running epoch: 2


Epoch: 2 , Running loss: 1.5181000323295593
Running epoch: 3


Epoch: 3 , Running loss: 1.1902403931617738
Running epoch: 4


Epoch: 4 , Running loss: 0.9403363742828369


In [63]:
torch.save(model.state_dict(),'model/pytorch_model.bin')

In [26]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json
# url = https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json
# output_directory = 'model'
# filename = wget.download(url, out=output_directory)

--2022-05-04 20:05:36--  https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.131.8
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.131.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1199 (1.2K) [application/json]
Saving to: ‘t5-base-config.json’

t5-base-config.json 100%[===================>]   1.17K  --.-KB/s    in 0s      

2022-05-04 20:05:37 (50.6 MB/s) - ‘t5-base-config.json’ saved [1199/1199]



In [64]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model =T5ForConditionalGeneration.from_pretrained('model', 
                                                return_dict=True)
def generate(text,model,tokenizer):
   model.eval()
   input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), 
                               return_tensors="pt")  
   outputs = model.generate(input_ids)
   return tokenizer.decode(outputs[0])

In [65]:
generate("version 1.03, version 1.01 | Last version | List of fonts and files in Core fonts for the Web",model,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


'<pad> The fonts used for the Web site were the fonts released in the United Kingdom in 2001'

In [67]:
generate("Hey Ram, Hey Ram | Title | Filmography in Shruti Haasan filmography",model,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


'<pad> Shruti Haasan made the film "Hey Ram" in Hindi and Hey'